In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# own Modules 
from models import LstmMultiTaskLearning, AnalyseLayer
from data_loader import DataPreperator, DataSet
from trainer import TrainerMultiTaskLearning
from loss_module import LossModuleMse

## Take care of these things before training:
- Select correct path and define droped_features
- Change parameter of model
- Change step_size in cycling_lr
- Change filed_location

## Hyperparameters

In [2]:
param = {
    "data" : {
        "stake_training_data" : 0.75, 
        "path" : '../../../data/phm_data_challenge/01_M01_DC_preprocessed_grid_search.csv' ,
    },
    "preprocessing" : {
        "first_order_difference" : False,
        "droped_features": ["ID", "ongoing time", "up time", "RLU", "runnum"],
        "features_not_to_scale": ['FIXTURESHUTTERPOSITION_0.0', 'FIXTURESHUTTERPOSITION_1.0',
                                      'FIXTURESHUTTERPOSITION_2.0', 'FIXTURESHUTTERPOSITION_3.0',
                                      'FIXTURESHUTTERPOSITION_255.0']
    },
    "model" : {
        "input_size" : 21,
        "n_hidden_lstm" : 100,
        "sequence_size" : 50,
        "batch_size" : 8,
        "lstm_layer" : 2,
        "n_hidden_fc_pred": 50,
        "n_hidden_fc_ls": 5,
        "dropout_rate": 0.2
    },
    "cycling_lr" : {
        "scheduler_active" : True, 
        # step_size is the number of training iterations (total samples/batch_size) per half cycle. 
        # Authors suggest setting step_size 2-8 x training iterations in epoch.
        "step_size" : (6000/8)*2, 
        # Mode can be one of {triangular, triangular2, exp_range}
        "mode" : "triangular", 
        "gamma" : 0.9995,
        "base_lr" : 0.016, 
        "max_lr" :0.1
    },
    "training": {
        "n_epochs" : 100,
        "patience" : 10,
    },
    "filed_location": {
        "trained_model" : "../../../models/MSE_multi_learning/",
        "history" : "../../visualisation/files/history_training/multi_learning.csv",
        "latent_space" : "./latent_space.csv"
    }
}

## Preprocessing
1. First order difference (if selected)
2. Split data into train and validation data
3. Scale train and validation data with train's mean and variance

In [3]:
train_loader = DataPreperator(path=param['data']['path'], 
                              ignored_features=param['preprocessing']['droped_features'],
                              stake_training_data=param['data']['stake_training_data'],
                              features_not_to_scale=param['preprocessing']['features_not_to_scale'],
                              first_order_difference=param["preprocessing"]["first_order_difference"])
train_data, validation_data = train_loader.prepare_data()
print(len(train_data)+len(validation_data))

6251


### Mean and variance from scale process (only of continious features)

In [4]:
mean, var = train_loader.provide_statistics()
print(mean)
print(var)

[-0.00616403  0.0014136  -0.00775649  0.02678833  0.11496844  0.01260359
 -0.03450196  0.01716666  0.00792266  0.05113533  0.15137018 -0.15249734
 -0.06415179 -0.25143178  0.87424645 -0.30155094]
[9.54757947e-01 9.65299841e-01 9.79715208e-01 9.95428027e-01
 1.20228023e+00 9.84873713e-01 4.43684229e-01 1.01021270e+00
 9.70240421e-01 6.98134612e-01 1.44322416e+00 9.29744851e-12
 1.95742556e-05 1.38008019e-05 4.47681049e-05 1.81650562e-01]


## Initialize Dataset 
Time series data must be transformed into a structure of samples with `input` and `target` components before it can be used to fit a supervised learning model. <br>
For a time series interested in one-step predictions, the observations at prior time steps, so-called lag observations, are used as `input` and the `target` is the observation at the current time step.

For example, a univariate series can be expressed as a supervised learning problem with three time steps for `input` and one step as `target`, as follows:

|input|target|
|-----|------|
[1, 2, 3]|[4]
[2, 3, 4]|[5]
[3, 4, 5]|[6]

The Keras deep learning library provides the `TimeseriesGenerator` to automatically transform both univariate and multivariate time series data into such a format. 

In [5]:
dataset_train = DataSet(train_data, timesteps=param["model"]["sequence_size"])
dataset_validation = DataSet(validation_data, timesteps=param["model"]["sequence_size"])

## Initialize DataLoader
Actually the data has a other size than in the table above because of multivariate time series data and because of using batches. <br>
__First dimension__: batch size --> Defines the number of samples that will be propagated through the network simultaneously. <br>
__Second dimension:__ timesteps --> Number of sequence which is passed into the LSTM <br>
__Third dimension:__ input_dim --> Is the number of features. In this case data from 7 sensors, collected at the same time. <br>

![](../../../knowledge/pictures/input_shape.png)

Data is shuffled because each mini batch is indipendent from each other, but samples of a minibatch are in chronological order.

In [6]:
data_loader_training = DataLoader(dataset_train, 
                                  batch_size=param["model"]["batch_size"], 
                                  num_workers=4, 
                                  shuffle=True, 
                                  drop_last=True
                                 )
data_loader_validation = DataLoader(dataset_validation, 
                                    batch_size=param["model"]["batch_size"], 
                                    num_workers=4, 
                                    shuffle=True, 
                                    drop_last=True
                                   )

In [7]:
for epoch in range(2):
    for batch_idx, data in enumerate(data_loader_training):
        x,y = data
        print("Epoch: {}".format(epoch))
        print('Data of batch: {}'.format(batch_idx))
        print("Data of X: {}".format(x))
        if batch_idx >=0: break

Epoch: 0
Data of batch: 0
Data of X: tensor([[[ 0.5193, -0.1893,  0.0753,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.5193, -0.1899,  0.0774,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.5193, -0.1897,  0.0780,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.5193, -0.1898,  0.0782,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.5193, -0.1896,  0.0785,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.5193, -0.1901,  0.0772,  ...,  0.0000,  0.0000,  0.0000]],

        [[-0.3199, -1.0750, -1.1559,  ...,  0.0000,  0.0000,  0.0000],
         [-0.7148, -1.1488, -1.1559,  ...,  0.0000,  0.0000,  0.0000],
         [-0.8722, -1.1499, -1.1563,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [-1.3794, -1.1499, -1.1557,  ...,  0.0000,  0.0000,  0.0000],
         [-1.3794, -1.1499, -1.1556,  ...,  0.0000,  0.0000,  0.0000],
         [-1.3794, -1.1499, -1.1557,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.7403,  1.1028,  1.0972,  ...,  0.0000,  0.0000,  0.0000],
        

In [8]:
for batch_idx, data in enumerate(data_loader_training):
    x,y = data
    print('Data of batch: {}'.format(batch_idx))
    print("Size of input data: {}".format(x.size()))
    print("Size of target data: {}".format(y.size()))
    if batch_idx >=1: break

Data of batch: 0
Size of input data: torch.Size([8, 50, 21])
Size of target data: torch.Size([8, 21])
Data of batch: 1
Size of input data: torch.Size([8, 50, 21])
Size of target data: torch.Size([8, 21])


## Initialize Neural Network
__Parameters for LSTM Modul:__
- input_size : The number of expected features in the input x
- hidden_size :The number of features in the hidden state h
- num_layers : Number of recurrent layers. E.g., setting num_layers=2 would mean stacking two LSTMs together to form a stacked LSTM, with the second LSTM taking in outputs of the first LSTM and computing the final results.
- batch_first : If True, then the input __and output__ tensors are provided as (batch, seq, feature).
- dropout – If non-zero, introduces a Dropout layer on the outputs of each LSTM layer except the last layer, with dropout probability equal to dropout. Default: 0

In [9]:
torch.manual_seed(0)
model = LstmMultiTaskLearning(batch_size=param['model']['batch_size'], 
                              input_dim=param['model']['input_size'], 
                              n_hidden_lstm=param['model']['n_hidden_lstm'], 
                              n_layers=param['model']['lstm_layer'],
                              dropout_rate= param['model']['dropout_rate'],
                              n_hidden_fc_prediction=param['model']['n_hidden_fc_pred'], 
                              n_hidden_fc_ls_analysis=param['model']['n_hidden_fc_ls'], 
                              file_location_latent_space=param["filed_location"]["latent_space"]         
                              )

## Define MSE Loss function as torch module

In [10]:
criterion = LossModuleMse(param["model"]["input_size"], param["model"]["batch_size"])

## Initialize Optimizer and Cyclic Learning Rate Scheduler

In [11]:
optimizer = torch.optim.SGD(model.parameters(), lr=1.)  
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer=optimizer, 
                                              base_lr=param['cycling_lr']['base_lr'], 
                                              max_lr=param['cycling_lr']['max_lr'], 
                                              step_size_up=param['cycling_lr']['step_size'], 
                                              mode=param['cycling_lr']['mode'],
                                              gamma=param['cycling_lr']['gamma']
                                             )

## Initialize Trainer

In [12]:
trainer = TrainerMultiTaskLearning(model=model,
                                   optimizer=optimizer,
                                   scheduler=scheduler,
                                   scheduler_active = param["cycling_lr"]["scheduler_active"],
                                   criterion=criterion, 
                                   location_model=param["filed_location"]["trained_model"], 
                                   location_stats=param["filed_location"]["history"], 
                                   patience=param['training']['patience']
                                   )

## Training Loop
An epoch consists of a learning cycle over all batches of training data and an evaluation of the most recent model with the testing data. 

In [13]:
# Create lists to save training loss and validation loss of each epoch
hist_loss = []
torch.manual_seed(0)

for epoch in range(param['training']['n_epochs']):
    # Train with batches 
    mean_epoch_training_loss = trainer.train(data_loader_training)

    # Evaluate
    mean_epoch_validation_loss = trainer.evaluate(data_loader_validation, hist_loss, epoch)

    # Cache History
    trainer.cache_history_training(hist_loss, epoch, mean_epoch_training_loss, mean_epoch_validation_loss)

    # Save model if its the best one since the last change in configuration of hyperparameters
    status_ok = trainer.save_model(epoch, mean_epoch_validation_loss, param['model']['input_size'], 
                                   param['model']['lstm_layer'], param['model']['n_hidden_lstm'], 
                                   param['model']['n_hidden_fc_pred'], param["model"]["sequence_size"],
                                   param['model']['n_hidden_fc_ls']
                                  )
    if not status_ok:
        break

# Safe results to csv file
df = pd.DataFrame(hist_loss)
df.to_csv(param["filed_location"]["history"], sep=";", index=False)

-------- epoch_no. 0 finished with eval loss 1.2538799390433326--------
Epoch 0: best model saved with loss: 1.2538799390433326
-------- epoch_no. 1 finished with eval loss 1.0297310153051975--------
Epoch 1: best model saved with loss: 1.0297310153051975
-------- epoch_no. 2 finished with eval loss 0.9183272661697086--------
Epoch 2: best model saved with loss: 0.9183272661697086
-------- epoch_no. 3 finished with eval loss 0.8508673845322201--------
Epoch 3: best model saved with loss: 0.8508673845322201
-------- epoch_no. 4 finished with eval loss 0.8041733030288939--------
Epoch 4: best model saved with loss: 0.8041733030288939
-------- epoch_no. 5 finished with eval loss 0.7687423527109349--------
Epoch 5: best model saved with loss: 0.7687423527109349
-------- epoch_no. 6 finished with eval loss 0.7450863083969529--------
Epoch 6: best model saved with loss: 0.7450863083969529
-------- epoch_no. 7 finished with eval loss 0.7253599358752133--------
Epoch 7: best model saved with l

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/carlovoss/anaconda3/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/Users/carlovoss/anaconda3/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/Users/carlovoss/anaconda3/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/Users/carlovoss/anaconda3/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/Users/carlovoss/anaconda3/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/Users/carlovoss/anaconda3/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/Users/car

KeyboardInterrupt: 

## Visualization of cyclic learning rate

In [ ]:
x = range(len(lr_find_lr))
data = pd.DataFrame(data={'y': lr_find_lr, 'x': x})
f, ax = plt.subplots(figsize=(7, 7))
sns.lineplot(x=data.x, y=data.y, ax=ax)
plt.show()